<a href="https://colab.research.google.com/github/Vraj110/Big-Data-Apriori/blob/main/YAFIM_Apriori_Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet pyspark

     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 54.5 MB/s 


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("apriori").setMaster("local")
sc = SparkContext(conf=conf)

In [4]:
file = "/content/gdrive/My Drive/Datasets/Project/Transaction.csv"
lineRDD = sc.textFile(file)
RDD = lineRDD.map(lambda line : line.split(","))

In [5]:
import pandas as pd

tr = pd.read_csv(file)
tr.columns

Index(['bruises', 'gill-attachment', 'gill-close-spacing', 'gill-narrow-size',
       'enlarge-stalk-shape', 'poisonous', 'edible', 'convex_cap', 'bell_cap',
       'sunken_cap',
       ...
       'several_pop', 'solitary_pop', 'clustered_pop', 'urban', 'grasses',
       'meadows', 'woods', 'paths', 'waste', 'leaves'],
      dtype='object', length=113)

For counting Support

In [6]:
def Count(rdd):
  count = 0
  for i in rdd:
    if( i[0] == '1' ):
      count = count + 1

  return count

Length 1: Frequent Itemset

In [24]:
from collections import OrderedDict
import numpy as np

L_1 = {}
min_sup = 4000
for i in range(len(tr.columns)):
  rdd = RDD.map(lambda line: (line[i],1))
  count = Count(rdd.collect())
  if( count >= min_sup ):
    L_1[tr.columns[i]] = count
L_1 = (sorted(L_1.items()))
L_1

[('edible', 4208),
 ('gill-close-spacing', 6812),
 ('one_ring', 7488),
 ('partial_veil', 8124),
 ('several_pop', 4040),
 ('smooth_above_ring', 5176),
 ('smooth_below_ring', 4936),
 ('white_above_ring', 4464),
 ('white_below_ring', 4384),
 ('white_veil', 7924)]

Candidate Set Genereation

In [25]:
def Candidate_Set(k,list):
  Cand = []
  for i in list:
    for j in list:
      temp = True
      if( k > 2 and i[0] != j[0] ):
        for l in range(k-2):
          if(i[0][l] != j[0][l]):
            temp = False
            break
        if( temp == True ):
          l = []
          for a in range(k-1):
            l.append(i[0][a])
          l.append(j[0][k-2])
          Cand.append(sorted(l))
      else:
        l = []
        if( i[0] != j[0] ):
          l.append(i[0])
          l.append(j[0])
          Cand.append(sorted(l))
  Cand = sorted(Cand)
  res = [t for t in (set(tuple(i) for i in Cand))]
  res = sorted(res)
  return res

Candidate Pruning i.e. checking support > min_Sup

In [26]:
def Candidate_Pruning(RDD,list,k):
  can_pru = {}

  for j in list:
    can_pru[j] = 0

  for i in RDD:
    for j in list:
      count = 0
      for l in range(k):
        index = tr.columns.get_loc(j[l])
        if( i[index] == '1' ):
          count = count + 1
      if( count == k ):
        can_pru[j] = can_pru[j] + 1

  pruned = {}
  for i in can_pru.keys():
    if( can_pru[i] != 0 and can_pru[i] >= min_sup ):
      pruned[i] = can_pru[i]

  pruned = sorted(pruned.items())
  return pruned

Finding All Frequent Itemsets

In [19]:
Freq_List = [L_1]
k = 2

candidate_set = Candidate_Set(k,L_1)
candidate_pruned = Candidate_Pruning(RDD.collect(), candidate_set, k)
Freq_List.append(candidate_pruned)
k = k + 1

while(True):
  candidate_set = Candidate_Set(k,candidate_pruned)
  candidate_pruned = Candidate_Pruning(RDD.collect(), candidate_set, k)
  if( len(candidate_pruned) == 0 ):
    break
  else:
    Freq_List.append(candidate_pruned)
    k = k + 1

In [20]:
print(k)

6


In [ ]:
Freq_List

[[('bruises', 3376),
  ('bulbous', 3776),
  ('convex_cap', 3656),
  ('edible', 4208),
  ('enlarge-stalk-shape', 3516),
  ('evanescent_ring', 2776),
  ('flat_cap', 3152),
  ('gill-close-spacing', 6812),
  ('gill-narrow-size', 2512),
  ('none', 3528),
  ('one_ring', 7488),
  ('partial_veil', 8124),
  ('pendant_ring', 3968),
  ('poisonous', 3916),
  ('scaly_cap', 3244),
  ('several_pop', 4040),
  ('smooth_above_ring', 5176),
  ('smooth_below_ring', 4936),
  ('smooth_cap', 2556),
  ('white_above_ring', 4464),
  ('white_below_ring', 4384),
  ('white_veil', 7924),
  ('woods', 3148)],
 [(('bruises', 'edible'), 2752),
  (('bruises', 'gill-close-spacing'), 3272),
  (('bruises', 'one_ring'), 3080),
  (('bruises', 'partial_veil'), 3376),
  (('bruises', 'pendant_ring'), 3184),
  (('bruises', 'smooth_above_ring'), 3232),
  (('bruises', 'smooth_below_ring'), 3040),
  (('bruises', 'white_veil'), 3376),
  (('bulbous', 'gill-close-spacing'), 3528),
  (('bulbous', 'one_ring'), 3656),
  (('bulbous', 'par

Checking each and every possible length Frequent Itemset

In [27]:
can2 = Candidate_Set(2,L_1)
res2 = Candidate_Pruning(RDD.collect(), can2, 2)
can2

[('edible', 'gill-close-spacing'),
 ('edible', 'one_ring'),
 ('edible', 'partial_veil'),
 ('edible', 'several_pop'),
 ('edible', 'smooth_above_ring'),
 ('edible', 'smooth_below_ring'),
 ('edible', 'white_above_ring'),
 ('edible', 'white_below_ring'),
 ('edible', 'white_veil'),
 ('gill-close-spacing', 'one_ring'),
 ('gill-close-spacing', 'partial_veil'),
 ('gill-close-spacing', 'several_pop'),
 ('gill-close-spacing', 'smooth_above_ring'),
 ('gill-close-spacing', 'smooth_below_ring'),
 ('gill-close-spacing', 'white_above_ring'),
 ('gill-close-spacing', 'white_below_ring'),
 ('gill-close-spacing', 'white_veil'),
 ('one_ring', 'partial_veil'),
 ('one_ring', 'several_pop'),
 ('one_ring', 'smooth_above_ring'),
 ('one_ring', 'smooth_below_ring'),
 ('one_ring', 'white_above_ring'),
 ('one_ring', 'white_below_ring'),
 ('one_ring', 'white_veil'),
 ('partial_veil', 'several_pop'),
 ('partial_veil', 'smooth_above_ring'),
 ('partial_veil', 'smooth_below_ring'),
 ('partial_veil', 'white_above_ring')

In [ ]:
can3 = Candidate_Set(3,res2)
res3 = Candidate_Pruning(RDD.collect(), can3, 3)
res3

[(('bruises', 'edible', 'gill-close-spacing'), 2656),
 (('bruises', 'edible', 'one_ring'), 2528),
 (('bruises', 'edible', 'partial_veil'), 2752),
 (('bruises', 'edible', 'pendant_ring'), 2560),
 (('bruises', 'edible', 'smooth_above_ring'), 2752),
 (('bruises', 'edible', 'smooth_below_ring'), 2560),
 (('bruises', 'edible', 'white_veil'), 2752),
 (('bruises', 'gill-close-spacing', 'one_ring'), 2976),
 (('bruises', 'gill-close-spacing', 'partial_veil'), 3272),
 (('bruises', 'gill-close-spacing', 'pendant_ring'), 3080),
 (('bruises', 'gill-close-spacing', 'smooth_above_ring'), 3128),
 (('bruises', 'gill-close-spacing', 'smooth_below_ring'), 2936),
 (('bruises', 'gill-close-spacing', 'white_veil'), 3272),
 (('bruises', 'one_ring', 'partial_veil'), 3080),
 (('bruises', 'one_ring', 'pendant_ring'), 3080),
 (('bruises', 'one_ring', 'smooth_above_ring'), 2936),
 (('bruises', 'one_ring', 'smooth_below_ring'), 2744),
 (('bruises', 'one_ring', 'white_veil'), 3080),
 (('bruises', 'partial_veil', 'p

In [ ]:
can4 = Candidate_Set(4,res3)
res4 = Candidate_Pruning(RDD.collect(), can4, 4)
res4

[(('bruises', 'edible', 'gill-close-spacing', 'partial_veil'), 2656),
 (('bruises', 'edible', 'gill-close-spacing', 'smooth_above_ring'), 2656),
 (('bruises', 'edible', 'gill-close-spacing', 'white_veil'), 2656),
 (('bruises', 'edible', 'one_ring', 'partial_veil'), 2528),
 (('bruises', 'edible', 'one_ring', 'pendant_ring'), 2528),
 (('bruises', 'edible', 'one_ring', 'smooth_above_ring'), 2528),
 (('bruises', 'edible', 'one_ring', 'white_veil'), 2528),
 (('bruises', 'edible', 'partial_veil', 'pendant_ring'), 2560),
 (('bruises', 'edible', 'partial_veil', 'smooth_above_ring'), 2752),
 (('bruises', 'edible', 'partial_veil', 'smooth_below_ring'), 2560),
 (('bruises', 'edible', 'partial_veil', 'white_veil'), 2752),
 (('bruises', 'edible', 'pendant_ring', 'smooth_above_ring'), 2560),
 (('bruises', 'edible', 'pendant_ring', 'white_veil'), 2560),
 (('bruises', 'edible', 'smooth_above_ring', 'smooth_below_ring'), 2560),
 (('bruises', 'edible', 'smooth_above_ring', 'white_veil'), 2752),
 (('brui

In [ ]:
can5 = Candidate_Set(5,res4)
res5 = Candidate_Pruning(RDD.collect(), can5, 5)
res5

[(('bruises',
   'edible',
   'gill-close-spacing',
   'partial_veil',
   'smooth_above_ring'),
  2656),
 (('bruises', 'edible', 'gill-close-spacing', 'partial_veil', 'white_veil'),
  2656),
 (('bruises',
   'edible',
   'gill-close-spacing',
   'smooth_above_ring',
   'white_veil'),
  2656),
 (('bruises', 'edible', 'one_ring', 'partial_veil', 'pendant_ring'), 2528),
 (('bruises', 'edible', 'one_ring', 'partial_veil', 'smooth_above_ring'),
  2528),
 (('bruises', 'edible', 'one_ring', 'partial_veil', 'white_veil'), 2528),
 (('bruises', 'edible', 'one_ring', 'pendant_ring', 'smooth_above_ring'),
  2528),
 (('bruises', 'edible', 'one_ring', 'pendant_ring', 'white_veil'), 2528),
 (('bruises', 'edible', 'one_ring', 'smooth_above_ring', 'white_veil'), 2528),
 (('bruises', 'edible', 'partial_veil', 'pendant_ring', 'smooth_above_ring'),
  2560),
 (('bruises', 'edible', 'partial_veil', 'pendant_ring', 'white_veil'), 2560),
 (('bruises',
   'edible',
   'partial_veil',
   'smooth_above_ring',
  

In [ ]:
can6 = Candidate_Set(6,res5)
res6 = Candidate_Pruning(RDD.collect(), can6, 6)
res6

[(('bruises',
   'edible',
   'gill-close-spacing',
   'partial_veil',
   'smooth_above_ring',
   'white_veil'),
  2656),
 (('bruises',
   'edible',
   'one_ring',
   'partial_veil',
   'pendant_ring',
   'smooth_above_ring'),
  2528),
 (('bruises',
   'edible',
   'one_ring',
   'partial_veil',
   'pendant_ring',
   'white_veil'),
  2528),
 (('bruises',
   'edible',
   'one_ring',
   'partial_veil',
   'smooth_above_ring',
   'white_veil'),
  2528),
 (('bruises',
   'edible',
   'one_ring',
   'pendant_ring',
   'smooth_above_ring',
   'white_veil'),
  2528),
 (('bruises',
   'edible',
   'partial_veil',
   'pendant_ring',
   'smooth_above_ring',
   'white_veil'),
  2560),
 (('bruises',
   'edible',
   'partial_veil',
   'smooth_above_ring',
   'smooth_below_ring',
   'white_veil'),
  2560),
 (('bruises',
   'gill-close-spacing',
   'one_ring',
   'partial_veil',
   'pendant_ring',
   'smooth_above_ring'),
  2832),
 (('bruises',
   'gill-close-spacing',
   'one_ring',
   'partial_veil

In [ ]:
can7 = Candidate_Set(7,res6)
res7 = Candidate_Pruning(RDD.collect(), can7, 7)
res7

[(('bruises',
   'edible',
   'one_ring',
   'partial_veil',
   'pendant_ring',
   'smooth_above_ring',
   'white_veil'),
  2528),
 (('bruises',
   'gill-close-spacing',
   'one_ring',
   'partial_veil',
   'pendant_ring',
   'smooth_above_ring',
   'smooth_below_ring'),
  2568),
 (('bruises',
   'gill-close-spacing',
   'one_ring',
   'partial_veil',
   'pendant_ring',
   'smooth_above_ring',
   'white_veil'),
  2832),
 (('bruises',
   'gill-close-spacing',
   'one_ring',
   'partial_veil',
   'pendant_ring',
   'smooth_below_ring',
   'white_veil'),
  2640),
 (('bruises',
   'gill-close-spacing',
   'one_ring',
   'partial_veil',
   'smooth_above_ring',
   'smooth_below_ring',
   'white_veil'),
  2568),
 (('bruises',
   'gill-close-spacing',
   'one_ring',
   'pendant_ring',
   'smooth_above_ring',
   'smooth_below_ring',
   'white_veil'),
  2568),
 (('bruises',
   'gill-close-spacing',
   'partial_veil',
   'pendant_ring',
   'smooth_above_ring',
   'smooth_below_ring',
   'white_ve

In [ ]:
can8 = Candidate_Set(8,res7)
res8 = Candidate_Pruning(RDD.collect(), can8, 8)
res8

[(('bruises',
   'gill-close-spacing',
   'one_ring',
   'partial_veil',
   'pendant_ring',
   'smooth_above_ring',
   'smooth_below_ring',
   'white_veil'),
  2568)]

In [ ]:
can9 = Candidate_Set(9,res8)
res9 = Candidate_Pruning(RDD.collect(), can9, 9)
res9

[]

In [28]:
i = tr.columns.get_loc("edible")
i

6